In [1]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, \
    Dropout, Conv2DTranspose, Cropping2D, Add, UpSampling2D, BatchNormalization
from keras.layers.merge import concatenate
from image_segmentation_keras.keras_segmentation.models.model_utils import get_segmentation_model
from glob import glob
import sys
sys.path.insert(1, './src')
from crfrnn_model import get_crfrnn_model_def
from crfrnn_layer import CrfRnnLayer
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope
i=0
if i==0:
    input_height = 256
    input_width = 256
    n_classes = 3
    channels = 3
    
    img_input = Input(shape=(input_height,input_width, channels))

    conv0 = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
#     conv0 = Dropout(0.2)(conv0)
    conv0 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv0)
    bn0 = BatchNormalization()(conv0)
    pool0 = MaxPooling2D((2, 2))(bn0)
    
    conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool0)
#     conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv1)
    bn1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(bn1)

    conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool1)
#     conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv2)
    bn2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(bn2)

    conv3 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool2)
#     conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv3)
    bn3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D((2, 2))(bn3)
    
    conv4 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool3)
#     conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv4)
    print("conv4",conv4.shape)
    print('conv3',conv3.shape)

    up_= Conv2DTranspose(512,(2,2),strides=2,padding='same')(conv4)
    print('up_',up_.shape)
    up0 = concatenate([up_, conv3], axis=3)
    print(up0.shape)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(up0)
#     conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    bn4 = BatchNormalization()(conv5)
    
    up_2= Conv2DTranspose(256,(2,2),strides=2,padding='same')(bn4)
    up1 = concatenate([up_2, conv2], axis=-1)
    print(up1.shape)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
#     conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    bn5 = BatchNormalization()(conv6)
    
    up_3= Conv2DTranspose(128,(2,2),strides=2,padding='same')(bn5)
    up2 = concatenate([up_3, conv1], axis=3)
    print(up2.shape)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
#     conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    bn6 = BatchNormalization()(conv7)
    
    up_4= Conv2DTranspose(64,(2,2),strides=2,padding='same')(bn6)
    up3 = concatenate([up_4, conv0], axis=3)
    print(up3.shape)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
#     conv8 = Dropout(0.2)(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    bn7 = BatchNormalization()(conv8)
    print(conv8.shape)
    out = Conv2D( n_classes, (1, 1), activation='relu', padding='same')(bn7)   
    crf_output = CrfRnnLayer(image_dims=(input_height, input_width),
                         num_classes=n_classes,
                         theta_alpha=160.,
                         theta_beta=3.,
                         theta_gamma=3.,
                         num_iterations=10,
                         name='crfrnn')([out, img_input])
    model = get_segmentation_model(img_input, crf_output)


Using TensorFlow backend.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1
conv4 (None, 16, 16, 1024)
conv3 (None, 32, 32, 512)
up_ (None, None, None, 512)
(None, 32, 32, 1024)
(None, 64, 64, 512)
(None, 128, 128, 256)
(None, 256, 256, 128)
(None, 256, 256, 64)


In [2]:
model.train(
    train_images =  "/Users/mavaylon/Research/Data1/train/img/",
    train_annotations = "/Users/mavaylon/Research/Data1/train/ann/",
    epochs=20,
    steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/train/img/*")),
    batch_size=1,
    validate=True,
    val_images="/Users/mavaylon/Research/Data1/test/img/",
    val_annotations="/Users/mavaylon/Research/Data1/test/ann/",
    val_batch_size=1,
    val_steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/test/img/*"))
)

Verifying training dataset


  2%|▏         | 33/1478 [00:00<00:04, 325.43it/s]

Dataset verified! 
Verifying validation dataset


100%|██████████| 1478/1478 [00:04<00:00, 338.55it/s]


Dataset verified! 
correct
Epoch 1/20
5912/5912 [==============================] - 17730s 3s/step - loss: 0.6694 - accuracy: 0.7411 - val_loss: 0.4883 - val_accuracy: 0.7627

Epoch 00001: val_accuracy improved from -inf to 0.76270, saving model to pet_class_crf.h5
Epoch 2/20
5912/5912 [==============================] - 17760s 3s/step - loss: 0.5798 - accuracy: 0.7819 - val_loss: 0.4677 - val_accuracy: 0.7896

Epoch 00002: val_accuracy improved from 0.76270 to 0.78955, saving model to pet_class_crf.h5
Epoch 3/20
5912/5912 [==============================] - 18053s 3s/step - loss: 0.5374 - accuracy: 0.7998 - val_loss: 0.4369 - val_accuracy: 0.8063

Epoch 00003: val_accuracy improved from 0.78955 to 0.80625, saving model to pet_class_crf.h5
Epoch 4/20
5912/5912 [==============================] - 18595s 3s/step - loss: 0.4879 - accuracy: 0.8200 - val_loss: 0.4273 - val_accuracy: 0.8040

Epoch 00004: val_accuracy did not improve from 0.80625
Epoch 5/20
5912/5912 [============================

In [3]:
model.load_weights('/Users/mavaylon/Research/pet_weights/unet_petcrf/unet__shuffled_pet_class_crf_bn_after_bothconv.h5')